##  HW3 - Flatting JSON data from Yelp




**Submission Instruction**

1- Replace black in the tile with your name.

2- Complete your notebook and run all the cells

3- Download .ipynb

4- Submit .ipynb file on Gradescope

**Setting Up Yelp API**

For this assignment we're going to be using the Yelp API via the python package `yelpapi`.  Like with the Spotify API, you need to go get a yelp developer account here: https://www.yelp.com/fusion. Here is step-by-step guide:

1- Go to https://www.yelp.com/fusion

2- Click on Start Free Trial and Click on Start Free 30-day Trial on Premium

3- Sign Up and Confirm your email address

4- After Signing Up you will see the Create New App page.

5- Put a name for your app, you can select Education as Industry. As your contact email put it a made up business email. I used riazi@microsoft.com. Finally select Premium as the Access Tier (without Premium access you cannot fetch reviews from Yelp and cannot complete this notebook). You can put NA under description.

6- Collect the write down your API Key. When you get the API key you can proceed with the homework.

The goal will be to make a dataset of local taco places that are doing well during Coronavirus. Specifically, we want to make a dataset that takes their average score since they've been open and compares it to the average score of the last three reviews.  Shops that have been doing well should hopefully have these two averages be similar.

There will be three main steps to this process:

* First you'll search by location type and get aggregate information for everything that falls in the ice cream category.
* Second you'll get reviews for all those locations.  Yelp only returns three reviews when you call an ID, but that still works. Given you can only query one ID at a time, you'll need to write a loop to create a dataframe of all the reviews.  
* Third, you'll aggregate the latest review information to see how their average review score compares to their overall average review score.  

In [ ]:
# Run this cell the first time, but after installing the yelpapi you need to reset the session. After reset skip this cell
# Mount google drive
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

In [ ]:
#Run this cell after the resetting the session
# Mount google drive
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'

sys.path.insert(0, nb_path)  # or append(nb_path)

In [ ]:
# Install yelpapi once. Next time you run this notebook, you can skip this.
!pip install --target=$nb_path yelpapi

In [ ]:
# Now enter your API key so you can make requests
from yelpapi import YelpAPI
from pandas import json_normalize
# yelp_api = YelpAPI('ENTER YOUR KEY HERE')
api_key = 'ENTER YOUR KEY HERE'
yelp_api = YelpAPI(api_key)


### Making calls to with Yelp API

There are many functions in the `yelpapi` package.  The first one we'll use is `search_query()`.  You can put in a term you want to search for followed up by the location and it'll give you all the locations that match.  For example, the following would search for all ice cream places here in Chicago.

```
ice = yelp_api.search_query(term = 'ice cream', location = 'Chicago, IL')
```

This will give you a response of all the ice cream places in Chicago. It'll be in JSON form so will need flattening before being useful.

### Getting all taco shops and flattening - [6 points]

**Task:** Start by making a dataframe that uses the `search_query()` function to search using the term 'taco'.  Call this `taco_shops`.  After that, flatten the json results to `taco_shops_df`.

In [ ]:
# search for taco shops and store to taco_shops
... = yelp_api.search_query(...,  ...)

In [ ]:
# Look at taco_shops
...

In [ ]:
# What keys are present?
...

Now use the json_normalize function to flatten `taco_shops`.  Note that you need to select the key that contains the businesses when flattening.  But this one is easier than the examples from the homework in that you don't need to provide any other arguments.  

Store the result as `taco_shops_df`

After that select only the columns 'id', 'alias', 'name', 'review_count', and 'rating'.

In [ ]:
# Flatten to taco_shops_df
taco_shops_df = ...
taco_shops_df.head() # Check

In [ ]:
# Select only necessary columns
... = ....[['...', '...', ...]]
taco_shops_df.shape # Check shape.  Is it 20 x 5?

## Getting reviews for the taco shops - [12 points]

Now we're going to use the `reviews_query()` function to get the last three reviews for a given ID.  The issue here is that you can only feed it one ID at a time.  So, we'll have to write a loop that queries for each ID in `taco_shops_df` and builds out a dataframe of reviews.  

**Task:**  Write a for loop that does the following steps:
* First make an empty list outside of the loop called `taco_shops_reviews_df`
* Initalize your loop so that it runs the length of `taco_shops_df` you made earlier.
* For each i in loop, use the id from `taco_shops_df` to get reviews from yelp using the `reviews_query()` function in `yelp_api` and store it to an object called `reviews`.
* Flatten `reviews` to an objected called `reviews_df`
* Add `location_id` to `reviews_df` - I gave you the code to do this :)
* Append `reviews_df` to `taco_shops_reviews_df`
* After the loop Transform `taco_shops_reviews_df` to a dataframe with the same name using concat
* Finally select only the columns `'id', 'text', 'rating', 'time_created', 'location_id'`
* Note: You need to slow down retreving the reviews using time.sleep(1)

In [ ]:
# Write our loop
.... = list()

for i in ...:

  ... = ...
  ... = ...
  reviews_df['location_id'] = taco_shops_df['id'][i]
  ... = ... ....(...)


In [ ]:
# Select columns 'id', 'text', 'rating', 'time_created', 'location_id'
...

### Aggregating your review data - [6 points]

**Task:** Now go and do a data aggregation to get the mean review score across the three reviews. Remember, we want this grouped by location_id.  Call this dataframe `latest_reviews_agg`.

In your groupby you should set `as_index` to false to make joining on the ID values easier.

You should also rename the second column to `rating_mean` vs. leaving it as the dual level name.


In [ ]:
# Do your groupby to get mean rating.
# Call it latest_reviews_agg.
... = ...(..., as_index=False).....

In [ ]:
# Check it!
...

In [ ]:
# Rename so the two column names are 'location_id' and 'mean_rating'
... = ...

### Join your two datasets and one last transform - [6 points]

**Task:** Now it's time to join `latest_reviews_agg` back to your `taco_shops_df` dataframe.  You're going to want to join them on the location_id, but remember it is called just 'id' in the `taco_shops_df`  dataframe.  Call this joined dataset `taco_shops_comp`

After you make that dataset do one last transform.  In this I want you to make a new column called 'still_good' where the value is 'yes' if the mean_rating is greater than or equal to the average rating since they opened, or 'no' if the rating has dropped.   The idea here is that this could be a value that one would use to see if their average score of the latest reviews has improved or suffered during Covid.  

In [ ]:
# Join and name taco_shops_comp

...

When you are comparing the columns make sure to not comparing taco_shops_df and taco_shops_comp since their tuples may have different orders! Use both columns from taco_shops_comp

In [ ]:
# Make still_good column
...

In [ ]:
# So, do any locations have a lower average in their last three reviews compared to their average overall score?
#Find the name of taco shops are not good anymore.

...